In [33]:
import pandas as pd
import math


df = pd.read_csv('Model Training\cleansed_df_train.csv')
df.head()

,AcceptedCmp1,AcceptedCmp2,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,Age,Complain,Education,Income,Kidhome,...,MntWines,NumCatalogPurchases,NumDealsPurchases,NumStorePurchases,NumWebPurchases,NumWebVisitsMonth,Recency,Teenhome,Time_Customer,Response
0,0,0,0,0,0,70,0,2n Cycle,62972,0,...,313,4,2,3,7,6,39,1,12.693151,0
1,0,0,0,0,0,65,0,Graduation,62807,0,...,526,3,3,12,5,5,83,1,12.616438,0
2,0,0,0,0,0,50,0,Graduation,76532,1,...,355,5,4,7,9,5,38,1,11.591781,0
3,0,0,0,0,0,39,0,2n Cycle,20491,0,...,5,1,1,2,0,7,16,0,12.213699,0
4,0,0,0,0,1,42,0,Graduation,78687,0,...,817,6,1,8,4,2,13,0,12.676712,1


In [2]:
# Verificar, para fins de análise, se há clientes que aceitaram mais de uma campanha
# Seleciona as colunas de campanhas aceitas
accepted_cols = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']

print(df[accepted_cols].sum(axis=1).value_counts())
multi_accepts = (df[accepted_cols].sum(axis=1) > 1).sum()
print(f"Clientes que aceitaram mais de uma campanha: {multi_accepts}")

0    1659
1     311
2      75
3      44
4      10
Name: count, dtype: int64
Clientes que aceitaram mais de uma campanha: 129


In [3]:
# Seleciona apenas as colunas categóricas (object ou category)
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
# Mostra os valores únicos de cada coluna categórica
for col in categorical_cols:
    print(f"\nColuna: {col}")
    print(df[col].unique())



Coluna: Education
['2n Cycle' 'Graduation' 'PhD' 'Master' 'Basic']

Coluna: Marital_Status
['Married' 'Together' 'Single' 'Divorced' 'Widow']


In [2]:
import pandas as pd
import math
df = pd.read_csv('Model Training\cleansed_df_train.csv')
df.head()

default_values = {
    "AcceptedCmp1": 0,
    "AcceptedCmp2": 0,
    "AcceptedCmp3": 0,
    "AcceptedCmp4": 0,
    "AcceptedCmp5": 0,
    "Response": 0,
    "Complain": 0,
    "Kidhome": 0,
    "Teenhome": 0,
    "NumCatalogPurchases": None,
    "NumDealsPurchases": None,
    "NumStorePurchases": None,
    "NumWebPurchases": None,
    "Education": 'Graduation',
    "Marital_Status": None,  # será definida como moda abaixo
    "Income": None,  # será definida como mediana
    "MntFishProducts": None,  # será definida como mediana
    "MntFruits": None,
    "MntGoldProds": None,
    "MntMeatProducts": None,
    "MntSweetProducts": None,
    "MntWines": None,
    "NumWebVisitsMonth": None,  # moda
    "Recency": None,  # mediana
    "Time_Customer": None,  # mediana
    "Age": None  # mediana
}

variaveis_info = {
    "AcceptedCmp1": ("booleana", "1 se cliente aceitou a oferta na 1ª campanha, 0 caso contrário", "0 ou 1"),
    "AcceptedCmp2": ("booleana", "1 se cliente aceitou a oferta na 2ª campanha, 0 caso contrário", "0 ou 1"),
    "AcceptedCmp3": ("booleana", "1 se cliente aceitou a oferta na 3ª campanha, 0 caso contrário", "0 ou 1"),
    "AcceptedCmp4": ("booleana", "1 se cliente aceitou a oferta na 4ª campanha, 0 caso contrário", "0 ou 1"),
    "AcceptedCmp5": ("booleana", "1 se cliente aceitou a oferta na 5ª campanha, 0 caso contrário", "0 ou 1"),
    "Response": ("booleana", "1 se cliente aceitou a última campanha, 0 caso contrário", "0 ou 1"),
    "Complain": ("booleana", "1 se reclamou nos últimos 2 anos, 0 caso contrário", "0 ou 1"),
    "DtCustomer": ("categórica", "Data de cadastro do cliente", "AAAA-MM-DD"),
    "Education": ("categórica", "Nível de escolaridade", "['2n Cycle', 'Graduation', 'PhD', 'Master', 'Basic']"),
    "Marital_Status": ("categórica", "Estado civil", "['Married', 'Together', 'Single', 'Divorced', 'Widow']"),
    "Kidhome": ("numérica", "Número de crianças pequenas no domicílio", ">= 0"),
    "Teenhome": ("numérica", "Número de adolescentes no domicílio", ">= 0"),
    "Income": ("numérica", "Renda anual do domicílio", ">= 0"),
    "MntFishProducts": ("numérica", "Valor gasto com peixes em 2 anos", ">= 0"),
    "MntMeatProducts": ("numérica", "Valor gasto com carnes em 2 anos", ">= 0"),
    "MntFruits": ("numérica", "Valor gasto com frutas em 2 anos", ">= 0"),
    "MntSweetProducts": ("numérica", "Valor gasto com doces em 2 anos", ">= 0"),
    "MntWines": ("numérica", "Valor gasto com vinhos em 2 anos", ">= 0"),
    "MntGoldProds": ("numérica", "Valor gasto com produtos de ouro em 2 anos", ">= 0"),
    "NumDealsPurchases": ("numérica", "Compras com desconto", ">= 0"),
    "NumCatalogPurchases": ("numérica", "Compras por catálogo", ">= 0"),
    "NumStorePurchases": ("numérica", "Compras em loja física", ">= 0"),
    "NumWebPurchases": ("numérica", "Compras online", ">= 0"),
    "NumWebVisitsMonth": ("numérica", "Visitas ao site no último mês", ">= 0"),
    "Recency": ("numérica", "Dias desde a última compra", ">= 0"),
    "Time_Customer": ("numérica", "Tempo (em dias) desde que se tornou cliente", ">= 0"),
    "Age": ("numérica", "Idade do cliente", ">= 0"),
}

desc = df.describe()

catalogo = []

for coluna in df.columns:
    if coluna == "Response":
        continue  # Ignorar essa coluna

    tipo, descricao, valores_esperados = variaveis_info.get(coluna, ("desconhecido", "Sem descrição", ""))

    if tipo == "numérica" and coluna in desc.columns:
        max_observado = df[coluna].max()
        folga_percentual = 2  # AJUSTAR

        max_com_folga = max_observado * (1 + folga_percentual)

        # Arredondar para cima com base no número de casas decimais
        if max_com_folga > 0:
            casas = int(math.log10(max_com_folga))  # número de casas antes da vírgula
            base = 10 ** casas
            max_val = int(math.ceil(max_com_folga / base) * base)
        else:
            max_val = 0

        # Limites fixos se fizer mais sentido
        if coluna == "Age":
            max_val = min(max_val, 120)
        elif coluna == "Recency":
            max_val = min(max_val, 365)

        valores_esperados = f">= 0 e <= {max_val}"

    # Determinar valor default
    if coluna in default_values and default_values[coluna] is not None:
        valor_default = default_values[coluna]
    elif tipo == "numérica":
        #if coluna in ["Income", "MntFishProducts", "MntFruits", "MntGoldProds", 'NumWebPurchases', 'NumStorePurchases','NumDealsPurchases', 'NumCatalogPurchases', 'Age', 'Teenhome', 'Kidhome',
         #             "MntMeatProducts", "MntSweetProducts", "MntWines", "Age","Recency", "Time_Customer"]:
        valor_default = round(df[coluna].median(), 2)
        #else:
         #   valor_default = 0
    elif tipo == "categórica":
        valor_default = df[coluna].mode().iloc[0] if not df[coluna].mode().empty else "N/A"
    elif tipo == "booleana":
        valor_default = 0
    else:
        valor_default = "N/A"

    tipo_python = df[coluna].dtype.name

    catalogo.append({
        "Variável": coluna,
        "Tipo": tipo,
        "Tipo Python": tipo_python,
        "Descrição": descricao,
        "Valores esperados": valores_esperados,
        "Valor default (caso ausente)": valor_default
    })

# Gerar DataFrame com o catálogo final
catalogo_df = pd.DataFrame(catalogo)
catalogo_df.to_csv('catalog.csv', index=False)
catalogo_df



,Variável,Tipo,Tipo Python,Descrição,Valores esperados,Valor default (caso ausente)
0,AcceptedCmp1,booleana,int64,"1 se cliente aceitou a oferta na 1ª campanha, ...",0 ou 1,0
1,AcceptedCmp2,booleana,int64,"1 se cliente aceitou a oferta na 2ª campanha, ...",0 ou 1,0
2,AcceptedCmp3,booleana,int64,"1 se cliente aceitou a oferta na 3ª campanha, ...",0 ou 1,0
3,AcceptedCmp4,booleana,int64,"1 se cliente aceitou a oferta na 4ª campanha, ...",0 ou 1,0
4,AcceptedCmp5,booleana,int64,"1 se cliente aceitou a oferta na 5ª campanha, ...",0 ou 1,0
5,Age,numérica,int64,Idade do cliente,>= 0 e <= 120,55.0
6,Complain,booleana,int64,"1 se reclamou nos últimos 2 anos, 0 caso contr...",0 ou 1,0
7,Education,categórica,object,Nível de escolaridade,"['2n Cycle', 'Graduation', 'PhD', 'Master', 'B...",Graduation
8,Income,numérica,int64,Renda anual do domicílio,>= 0 e <= 2000000,51563.0
9,Kidhome,numérica,int64,Número de crianças pequenas no domicílio,>= 0 e <= 6,0
